<a href="https://colab.research.google.com/github/hkolgur/UOH/blob/main/LSH_from_Scratch_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implement LSH from scratch

In this assignment, you will implement LSH from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you wirte and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% wieghtage in the final marks for this course.

## Reading the data from csv file

In [1]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
# Loading data from csv file
import pandas as pd
data_path = '/gdrive/My Drive/UOH/lsh/lsh_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...


In [3]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

#### Observation: we see all the categories have good amount of data points . The data set is balanced

### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [4]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [5]:
# For train_data here the labels are in the column named "category".
train_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,politics,teens know little of politics teenagers ques...
2211,entertainment,lopez misses uk charity premiere jennifer lope...
2212,business,christmas shoppers flock to tills shops all ov...
2213,tech,progress on new internet domains by early 2005...


In [6]:
test_data 

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


### Observation: There are 10 test questions for which the category is NaN on for which we have to predict the right category.

## Custom Implementation

### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **4000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate **5 random hyperplanes**. And generate the hyperplanes using a random normal distribution with **mean zero and variance 1**. 
  6. We have set the **numpy random seed to zero**, please do not change it. And then you can make use of **np.random.normal** to generate the vectors for hyperplanes.
  7. As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute **11-Nearest Neighbours** from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. **In case of a tie** in the obtained labels from nearest neighbours, you can pick a label after sorting all the labels **alphabetically**(A-Z), i.e. for example labels starting with A would get more preference than labels starting with Z.
  14. Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list you return should be of length 10.
  16. Also note that the cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
  17. Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.

## Grader Cell

Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.

In [7]:
from numpy.linalg import norm
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

#consider bigrams and trigrams, limit features to 4000 and min document frequency to 10
model=TfidfVectorizer(lowercase=True,ngram_range=(2,3),max_features=4000,min_df=10)

#train_vec will contain the vector representation of each of the row form training set 
train_vecs =model.fit_transform(train_data['text'].values.tolist())

vocab_size=train_vecs.get_shape()[1]  #final_model.get_shape()[1]
np.random.seed(0)
#Generate 5 Random Hyper Planes
n_pln=5
plane=[[0 for i in range(n_pln)] for j in range(vocab_size)]

#from a random normal distribution of mean 0 and std-dev 1 generate 5 planes.
#Keep the size of each plane equal to vocab size
for i in range(0,n_pln):
  plane[i]=np.random.normal(0,1,vocab_size)

#Key is a list of 5 dimensions, which is used to store info on which side of the plane does the train data element lie .
#Key is of size equal to training data size.
key=[[0 for i in range(n_pln)] for j in range(len(train_data))]


#Compute dot product between random plane and train vector based on which the key element is constructed
for i in range(0,len(train_data)):
  for j in range(0,n_pln):
    if np.dot(train_vecs[i].toarray().tolist()[0],plane[j].T)>=0:
    #if np.dot(model[i].toarray(),plane[j].T)>=0:
      key[i][j]=+1
    else:
      key[i][j]=-1
#define a dictionary to store all the train data vectors with same key as one item 
#Use the key generated above as key for the dictionary      
d=dict()
#Create a hash table with key as result of the bits from 5 hyperplanes for data point
# and value is each document. If its a new key add document in value field
# else if key already exists then append new document with same key as list 
  try:
    d[tuple(key[i])].append([train_vecs[i].toarray().tolist()[0],i])
  except KeyError:
    d[tuple(key[i])] = [[train_vecs[i].toarray().tolist()[0],i]]

In [8]:
# Please implement this fucntion and write your code wherever asked. Do NOT change the code snippets provided by us.

import numpy as np
import math
import collections

def predictLabels (test_data):
  """
  Given the test_data, return the labels for all the rows in the test data.
  Follow the step by step instructions mentioned above.
  """

  np.random.seed(0)
  #q_vect stores the vector representation of each of the test data element (total 10 items)
  q_vect=[]
  #list item to store cos of the angle between vectors 
  lst_cosine=[]
  #final_answer will have the final list of the category labels for which the query element belongs to after performing KNN
  final_answer=[]

  list_category=[]
  np.random.seed(0)
  for i,q_txt in enumerate(test_data['text']):
    lst_cosine.clear()
    list_category.clear()
    if i!=100: #This block is specifically to control on how many/which query point should we execute the prediction on
#convert the text version of the query point to vector
      q_vect=model.transform([q_txt])
#define 5 random hyperplanes to generate keys which will be later used to look up the dictionary constructed in training phase
      key_q=[0 for i in range(n_pln)]
      for j in range(0,n_pln):
        if np.dot(q_vect.toarray().tolist()[0],plane[j].T)>=0:
          key_q[j]=+1
        else:
          key_q[j]=-1

#Compute the cosine for each of the documents that have same key and find 11 best values
      for n,ele in enumerate(d[tuple(key_q)]):
        dt=np.dot(ele[0],q_vect.toarray().T)
        n1=norm(ele[0])
        n2=norm(q_vect.toarray()) 
#If n1=0 we are setting n1=1 to avoid Nan values 
        if (n1==0 or n2==0):
          n1=1
        lst_cosine.append([n,dt/(n1*n2),d[tuple(key_q)][n][1],train_data.iloc[d[tuple(key_q)][n][1]].category])
#sort the cosine values obtained form dot product of query with same key as test data in descending order 
      lst_cosine.sort(key = lambda x: x[1].tolist(),reverse=True)
      #print("top 11:",lst_cosine[0:11])
      for i in range(0,11):
        list_category.append(lst_cosine[0:11][i][3]) # add top 11 categories
#counter to maintain the category name and number of times it occured 
      c=collections.Counter(list_category)
#ctr contains the count related to higest number of times the category that was repeated
      ctr=c.most_common()[0][1]
#cat has the category value that was repeaed maximum number of times
      cat=c.most_common()[0][0]
#sub_lst is to sort the category names in ascending order when there is a tie on the number of times a category is voted
      sub_lst=[]
#If top 11 KNN has a tie , consider the category that comes in ascending order of the category names
      for ele in c:
        if c[ele]==ctr:
         sub_lst.append(ele)
      if len(sub_lst)>0:
        sub_lst.sort()
        cat=sub_lst[0]
    final_answer.append(cat)

  return final_answer
  ##############################################################
  ####  Write YOUR CODE BELOW  as per the above instructions ###
  ##############################################################




In [9]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
  print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
  print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
  print("\nY_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\nY_custom = \n\n", Y_custom)


******** Success ******** Accuracy Achieved =  90 %
